# Atividade 2.1 - Zero reais de funções reais

**Curso:** Bacharelado em Ciência da Computação  
**Disciplina:** Matemática Computacional  
**Professor:** Claudomiro de Souza de Sales Junior
**Aluno:** Alessandro Reali Lopes Silva

---

## Esse Nootebook tem como objetivo apresentar a resolução da Atividade 2.1 - Zero reais de funções reais.
1. Zero reais de funções reais
2. Estudo Especial de Equações Polinomiais

In [ ]:
from typing import Callable, Dict, Tuple, List, Any, Union
from IPython.display import display, Markdown
import math
import time
from dataclasses import dataclass, asdict

# Estrutura para os dados de CADA iteração (Histórico)
@dataclass
class Iteracao:
    iteracao: int
    x: float          # x_meio ou aproximação atual
    fx: float         # valor da função
    erro: float       # b - a ou |x_novo - x_velho|
    tempo: float      # ms
    operacoes: int
    decisoes: int

# Estrutura para o Resumo Final (Comparação)
@dataclass
class Resultado:
    metodo: str
    raiz: float
    f_raiz: float
    erro_final: float
    iteracoes_totais: int
    tempo_total: float
    ops_totais: int
    decisoes_totais: int

# --- FUNÇÃO AUXILIAR DE FORMATAÇÃO (CORRIGIDA) ---
def gerar_tabela_markdown(dados: List[Union[Iteracao, Resultado, Dict[str, Any]]]) -> None:
    """
    Gera uma tabela formatada em Markdown a partir de uma lista de Dataclasses OU Dicionários.
    """
    if not dados:
        display(Markdown("**Nenhum dado disponível para exibir.**"))
        return
    
    # Converte dataclasses para dicionários, se necessário
    if hasattr(dados[0], '__dataclass_fields__'):
        dados_processados = [asdict(d) for d in dados]
    else:
        dados_processados = dados # Já são dicionários

    # Pega as chaves (colunas) do primeiro item
    chaves = list(dados_processados[0].keys())
    
    # Cria o cabeçalho
    md = f"| {' | '.join(chaves)} |\n"
    md += f"| {' | '.join([':---:'] * len(chaves))} |\n" # Centraliza
    
    # Preenche as linhas
    for linha in dados_processados:
        valores_formatados = []
        for k in chaves:
            val = linha[k]
            
            # Formatação inteligente
            if isinstance(val, float):
                # Critério para notação científica: muito pequeno ou muito grande
                if (abs(val) < 1e-4 and val != 0) or abs(val) > 1e6:
                    valores_formatados.append(f"{val:.4e}")
                else:
                    valores_formatados.append(f"{val:.6f}")
            else:
                valores_formatados.append(str(val))
                
        md += f"| {' | '.join(valores_formatados)} |\n"
    
    display(Markdown(md))

In [ ]:
def metodo_bissecao(
    f: Callable[[float], float],
    x_min: float,
    x_max: float,
    epsilon: float,
) -> Tuple[Resultado, List[Iteracao]]:
    """
    Implementa o método da bisseção com rastreamento detalhado de esforço computacional.
    """
    # --- Inicialização Global ---
    operacoes = 0
    decisoes = 0
    chamadas_f = 0

    # Validação Inicial
    y_min = f(x_min)
    y_max = f(x_max)
    chamadas_f += 2
    
    operacoes += 1 # Multiplicação
    decisoes += 1  # If
    if y_min * y_max >= 0:
        raise ValueError("f(x_min) e f(x_max) devem ter sinais opostos.")
    
    historico: List[Iteracao] = []
    iteracoes = 0
    
    erro = (x_max - x_min)
    operacoes += 1 # Subtração

    tempo_inicial = time.perf_counter()

    while erro > epsilon:
        iteracoes += 1  # Conta a iteração
        tempo_iteracao_inicial = time.perf_counter()
        
        # Reinicia contadores locais da iteração
        decisoes_iteracao = 1 # Já conta o check do while
        operacoes_iteracao = 0
        
        x_meio = (x_min + x_max) / 2
        operacoes_iteracao += 2 # Soma e divisão

        y_meio = f(x_meio)
        chamadas_f += 1 # Global, pois conta no total
        
        # --- Lógica de Atualização do Intervalo ---
        decisoes_iteracao += 1 # If y_meio == 0
        raiz_exata_encontrada = False
        
        if y_meio == 0:
            raiz_exata_encontrada = True
        else:
            operacoes_iteracao += 1 # Multiplicação teste sinal
            decisoes_iteracao += 1  # If/Else
            
            if y_min * y_meio < 0:
                x_max = x_meio
            else:
                x_min = x_meio
                y_min = y_meio

        # Atualiza o erro para a tabela
        erro = (x_max - x_min)
        operacoes_iteracao += 1
        
        # Atualiza acumuladores globais com o esforço desta rodada
        operacoes += operacoes_iteracao
        decisoes += decisoes_iteracao
        
        # --- Salvamento do Histórico ---
        tempo_passado = (time.perf_counter() - tempo_iteracao_inicial) * 1000
        
        historico.append(Iteracao(
            iteracao=iteracoes,
            x=x_meio,
            fx=y_meio,
            erro=erro,
            tempo=tempo_passado,
            operacoes=operacoes_iteracao,
            decisoes=decisoes_iteracao,
        ))
        
        # Se achou a raiz exata, finaliza o loop
        if raiz_exata_encontrada:
            break

    tempo_final = time.perf_counter()
    tempo_total = (tempo_final - tempo_inicial) * 1000 

    # --- Criação do Resultado Final ---
    resultado = Resultado(
        metodo="Bisseção",
        raiz=x_meio,
        f_raiz=y_meio,
        erro_final=erro,
        iteracoes_totais=iteracoes,
        tempo_total=tempo_total,
        ops_totais=operacoes,
        decisoes_totais=decisoes,
    )
    
    return resultado, historico


def metodo_falsa_posicao(
    f: Callable[[float], float],
    x_min: float,
    x_max: float,
    epsilon: float,
) -> Tuple[Resultado, List[Iteracao]]:
    """
    Implementa o Método da Falsa Posição (Regula Falsi).
    Similar à bisseção, mas usa a média ponderada (pesos) pelos valores da função.
    """
    # --- Inicialização Global ---
    operacoes = 0
    decisoes = 0
    chamadas_f = 0

    # 1. Validação e Setup Inicial
    y_min = f(x_min)
    y_max = f(x_max)
    chamadas_f += 2
    
    operacoes += 1 # Multiplicação teste sinal
    decisoes += 1  # If teste sinal
    
    if y_min * y_max >= 0:
        raise ValueError("f(x_min) e f(x_max) devem ter sinais opostos.")
    
    historico: List[Iteracao] = []
    iteracoes = 0
    
    erro = (x_max - x_min)
    operacoes += 1

    tempo_inicial = time.perf_counter()

    while erro > epsilon:
        tempo_iteracao_inicial = time.perf_counter()
        
        decisoes_iteracao = 1 # Check do while
        operacoes_iteracao = 0
        iteracoes += 1

        # --- A Grande Diferença: A Fórmula da Posição ---
        # x = (x_min * f(x_max) - x_max * f(x_min)) / (f(x_max) - f(x_min))
        numerador = (x_min * y_max) - (x_max * y_min)
        denominador = (y_max - y_min)
        
        # Proteção contra divisão por zero (raro, mas possível se y_max == y_min)
        decisoes_iteracao += 1
        if denominador == 0:
            raise ValueError("Divisão por zero na fórmula da Falsa Posição.")
        
        x_falso = numerador / denominador
        
        # Contagem da fórmula: 2 mult, 2 sub, 1 div
        operacoes_iteracao += 5 

        # Avaliação da Função no novo ponto
        y_falso = f(x_falso)
        chamadas_f += 1 # Global
        
        # --- Lógica de Atualização (Igual Bisseção) ---
        decisoes_iteracao += 1 # If y_falso == 0
        raiz_exata = False

        if y_falso == 0:
            raiz_exata = True
        else:
            operacoes_iteracao += 1 # Multiplicação teste
            decisoes_iteracao += 1  # If/Else
            
            if y_min * y_falso < 0:
                x_max = x_falso
                y_max = y_falso
            else:
                x_min = x_falso
                y_min = y_falso

        # Atualiza erro
        erro = abs(x_max - x_min)
        operacoes_iteracao += 1 # Subtração        

        # Atualiza globais
        operacoes += operacoes_iteracao
        decisoes += decisoes_iteracao

        # Salva Histórico
        tempo_passado = (time.perf_counter() - tempo_iteracao_inicial) * 1000
        
        historico.append(Iteracao(
            iteracao=iteracoes,
            x=x_falso,
            fx=y_falso,
            erro=erro,
            tempo=tempo_passado,
            operacoes=operacoes_iteracao,
            decisoes=decisoes_iteracao,
        ))
        
        if raiz_exata:
            break

    tempo_final = time.perf_counter()
    tempo_total = (tempo_final - tempo_inicial) * 1000

    resultado = Resultado(
        metodo="Falsa Posição",
        raiz=x_falso,
        f_raiz=y_falso,
        erro_final=erro,
        iteracoes_totais=iteracoes,
        tempo_total=tempo_total,
        ops_totais=operacoes,
        decisoes_totais=decisoes,
    )
    
    return resultado, historico
    

def metodo_ponto_fixo(
    f: Callable[[float], float],
    phi: Callable[[float], float],
    x0: float,
    epsilon: float,
    max_iter: int = 100
) -> Tuple[Resultado, List[Iteracao]]:
    """
    Implementa o Método do Ponto Fixo (MPF).
    Requer a função de iteração 'phi' tal que x = phi(x).
    """
    # --- Inicialização Global ---
    operacoes = 0
    decisoes = 0
    chamadas_f = 0 # Contaremos chamadas para f() e phi()

    historico: List[Iteracao] = []
    iteracoes = 0
    x_atual = x0
    erro = float('inf')

    tempo_inicial = time.perf_counter()

    while erro > epsilon and iteracoes < max_iter:
        tempo_iteracao_inicial = time.perf_counter()
        
        decisoes_iteracao = 1 # Check do while
        operacoes_iteracao = 0
        iteracoes += 1

        # --- O Passo do MPF ---
        # A mágica acontece aqui: x(k+1) = phi(x(k))
        x_novo = phi(x_atual)
        chamadas_f += 1 

        # Avaliamos f(x) apenas para ver quão longe do zero estamos
        f_x_novo = f(x_novo)
        chamadas_f += 1

        # Cálculo do Erro (Critério de parada: distância entre x_novo e x_atual)
        erro = abs(x_novo - x_atual)
        operacoes_iteracao += 1 # Subtração

        # Atualiza globais
        operacoes += operacoes_iteracao
        decisoes += decisoes_iteracao

        # Salva Histórico
        tempo_passado = (time.perf_counter() - tempo_iteracao_inicial) * 1000
        
        historico.append(Iteracao(
            iteracao=iteracoes,
            x=x_novo,
            fx=f_x_novo,
            erro=erro,
            tempo=tempo_passado,
            operacoes=operacoes_iteracao,
            decisoes=decisoes_iteracao,
        ))

        # Atualiza para a próxima volta
        x_atual = x_novo
        
        # O MPF diverge muito fácil, então é bom ter um check de segurança
        decisoes += 1
        if erro > 1e6: # Se o erro explodiu para infinito
            print("Aviso: MPF Divergiu!")
            break

    tempo_final = time.perf_counter()
    tempo_total = (tempo_final - tempo_inicial) * 1000

    resultado = Resultado(
        metodo="Ponto Fixo",
        raiz=x_atual,
        f_raiz=f(x_atual),
        erro_final=erro,
        iteracoes_totais=iteracoes,
        tempo_total=tempo_total,
        ops_totais=operacoes,
        decisoes_totais=decisoes,
    )
    
    return resultado, historico


def metodo_newton(
    f: Callable[[float], float],
    df: Callable[[float], float],
    x0: float,
    epsilon: float,
    max_iter: int = 100
) -> Tuple[Resultado, List[Iteracao]]:
    """
    Implementa o método de Newton-Raphson com rastreamento detalhado de esforço computacional.
    """
    # --- Inicialização Global ---
    operacoes = 0
    decisoes = 0
    chamadas_f = 0 # Contaremos f(x) e f'(x) como chamadas

    historico: List[Iteracao] = []
    iteracoes = 0
    x_n = x0
    erro = float('inf')
    
    tempo_inicial = time.perf_counter()

    # Loop principal
    while erro > epsilon and iteracoes < max_iter:
        tempo_iteracao_inicial = time.perf_counter()
        
        # Reinicia contadores locais
        decisoes_iteracao = 1 # Check do while
        operacoes_iteracao = 0
        iteracoes += 1

        # 1. Avaliações (Função e Derivada)
        f_xn = f(x_n)
        df_xn = df(x_n)
        chamadas_f += 2 # Uma para f, uma para df
        
        # 2. Verificação Crítica (Derivada Zero)
        decisoes_iteracao += 1
        if df_xn == 0:
            # Salvamos o que temos até agora antes de falhar
            raise ValueError(f"Derivada zero encontrada na iteração {iteracoes}. O método falhou.")

        # 3. Passo de Newton: x(n+1) = x(n) - f(x)/f'(x)
        x_n1 = x_n - (f_xn / df_xn)
        operacoes_iteracao += 2 # Uma divisão, uma subtração

        # 4. Cálculo do Erro
        erro = abs(x_n1 - x_n)
        operacoes_iteracao += 1 # Subtração

        # Atualiza acumuladores globais
        operacoes += operacoes_iteracao
        decisoes += decisoes_iteracao

        # --- Salvamento do Histórico ---
        tempo_passado = (time.perf_counter() - tempo_iteracao_inicial) * 1000
        
        historico.append(Iteracao(
            iteracao=iteracoes,
            x=x_n,       # Mostramos o X usado nesta iteração
            fx=f_xn,     # O valor da função neste X
            erro=erro,   # O erro gerado para o próximo passo
            tempo=tempo_passado,
            operacoes=operacoes_iteracao,
            decisoes=decisoes_iteracao,
        ))

        # Atualiza x para a próxima rodada
        x_n = x_n1
        
        # Critério de parada extra: Raiz exata
        decisoes += 1 # Check do if
        if f_xn == 0:
            break

    tempo_final = time.perf_counter()
    tempo_total = (tempo_final - tempo_inicial) * 1000

    # --- Criação do Resultado Final ---
    # Nota: x_n agora contém o valor da última iteração (x_n1 da volta anterior)
    f_final = f(x_n)
    chamadas_f += 1 
    
    resultado = Resultado(
        metodo="Newton",
        raiz=x_n,
        f_raiz=f_final, 
        erro_final=erro,
        iteracoes_totais=iteracoes,
        tempo_total=tempo_total,
        ops_totais=operacoes,
        decisoes_totais=decisoes,
    )

    return resultado, historico

def metodo_secante(
    f: Callable[[float], float],
    x0: float,
    x1: float,
    epsilon: float,
    max_iter: int = 100
) -> Tuple[Resultado, List[Iteracao]]:
    """
    Implementa o método da secante com rastreamento detalhado de esforço computacional.
    """
    # --- Inicialização Global ---
    operacoes = 0
    decisoes = 0
    chamadas_f = 0 

    # 1. Pré-cálculo
    f_x0 = f(x0)
    f_x1 = f(x1)
    chamadas_f += 2
    
    historico: List[Iteracao] = []
    iteracoes = 0
    erro = float('inf')
    
    tempo_inicial = time.perf_counter()

    # Loop principal
    while erro > epsilon and iteracoes < max_iter:
        tempo_iteracao_inicial = time.perf_counter()
        
        # Reinicia contadores locais
        decisoes_iteracao = 1 # Check do while
        operacoes_iteracao = 0
        iteracoes += 1

        # 2. Verificação Crítica (Denominador Zero)
        # O denominador é (f(x1) - f(x0))
        denominador = f_x1 - f_x0
        operacoes_iteracao += 1 # Subtração
        
        decisoes_iteracao += 1
        if denominador == 0:
            raise ValueError(f"Divisão por zero na iteração {iteracoes}. f(x1) é igual a f(x0).")

        # 3. Passo da Secante: x2 = x1 - (f(x1) * (x1 - x0) / denominador)
        x2 = x1 - (f_x1 * (x1 - x0) / denominador)
        operacoes_iteracao += 4 # 1 sub (x1-x0), 1 mult, 1 div, 1 sub final

        # 4. Cálculo do Erro
        erro = abs(x2 - x1)
        operacoes_iteracao += 1 # Subtração/Abs

        f_x2 = f(x2)
        chamadas_f += 1

        # Atualiza acumuladores globais
        operacoes += operacoes_iteracao
        decisoes += decisoes_iteracao

        # --- Salvamento do Histórico ---
        tempo_passado = (time.perf_counter() - tempo_iteracao_inicial) * 1000
        
        historico.append(Iteracao(
            iteracao=iteracoes,
            x=x2,
            fx=f_x2,
            erro=erro,
            tempo=tempo_passado,
            operacoes=operacoes_iteracao,
            decisoes=decisoes_iteracao,
        ))
        
        # Critério de sorte (Raiz exata)
        decisoes += 1
        if f_x2 == 0:
            x1 = x2 # Para o resultado final apontar certo
            f_x1 = f_x2
            break

        # Prepara a próxima iteração (Dança das Cadeiras)
        # x0 assume o lugar de x1, e x1 assume o lugar de x2
        x0 = x1
        f_x0 = f_x1
        
        x1 = x2
        f_x1 = f_x2

    tempo_final = time.perf_counter()
    tempo_total = (tempo_final - tempo_inicial) * 1000 

    resultado = Resultado(
        metodo="Secante",
        raiz=x1,
        f_raiz=f_x1,
        erro_final=erro,
        iteracoes_totais=iteracoes,
        tempo_total=tempo_total,
        ops_totais=operacoes,
        decisoes_totais=decisoes,
    )

    return resultado, historico

### Zero reais de funções reais


In [ ]:
# --- EXEMPLO 18 ---
# f(x) = e^(-x^2) - cos(x)
# Raízes prováveis: x=0 e outra próxima de 1.44

def f18(x: float) -> float:
    """Função exemplo 18: f(x) = e^(-x^2) - cos(x)"""
    return math.exp(-(x**2)) - math.cos(x)

def df18(x: float) -> float:
    """
    Derivada f'(x):
    d/dx(e^(-x^2)) = -2x * e^(-x^2)
    d/dx(-cos(x))  = sin(x)
    """
    return -2 * x * math.exp(-(x**2)) + math.sin(x)

def phi_f18(x: float) -> float:
    """
    Função phi para o MPF.
    Isolando x em cos(x) = e^(-x^2) -> x = arccos(e^(-x^2))
    Nota: Converge para a raiz positiva se x0 for adequado.
    """
    try:
        val = math.exp(-(x**2))
        # Proteção matemática: arccos só aceita valor entre -1 e 1
        if val > 1: val = 1
        return math.acos(val)
    except ValueError:
        return x # Retorna o próprio x em caso de erro para não quebrar o loop

# --- EXEMPLO 19 ---
# f(x) = x^3 - x - 1
# Raiz provável: ~1.32

def f19(x: float) -> float:
    """Função exemplo 19: f(x) = x^3 - x - 1"""
    return x**3 - x - 1

def df19(x: float) -> float:
    """Derivada f'(x) = 3x^2 - 1"""
    return 3 * (x**2) - 1

def phi_f19(x: float) -> float:
    """
    Função phi para o MPF.
    Isolando x do termo cúbico: x^3 = x + 1 -> x = (x+1)^(1/3)
    Esta forma geralmente converge (derivada < 1 perto da raiz).
    """
    # math.pow funciona melhor com bases positivas.
    if x + 1 < 0:
        return -((- (x + 1))**(1/3)) # Tratamento para raiz cúbica de negativo
    return (x + 1)**(1/3)

# --- EXEMPLO 20 ---
# f(x) = 4sin(x) - e^x
# Raiz provável: ~0.37

def f20(x: float) -> float:
    """Função exemplo 20: f(x) = 4sin(x) - e^x"""
    return 4 * math.sin(x) - math.exp(x)

def df20(x: float) -> float:
    """Derivada f'(x) = 4cos(x) - e^x"""
    return 4 * math.cos(x) - math.exp(x)

def phi_f20(x: float) -> float:
    """
    Função phi para o MPF.
    Isolando x do seno: 4sin(x) = e^x -> sin(x) = e^x / 4 -> x = arcsin(e^x / 4)
    """
    try:
        val = math.exp(x) / 4
        # Proteção: arcsin só aceita entre -1 e 1
        if val > 1 or val < -1: 
            return x # Evita erro matemático mantendo o valor
        return math.asin(val)
    except ValueError:
        return x

# --- EXEMPLO 21 ---
# f(x) = x * ln(x) - 1
# Raiz provável: ~1.76

def f21(x: float) -> float:
    """Função exemplo 21: f(x) = x * ln(x) - 1"""
    # Proteção: log(x) só existe para x > 0
    if x <= 0: return -1e9 # Valor dummy para indicar erro
    return x * math.log(x) - 1

def df21(x: float) -> float:
    """
    Derivada f'(x): Regra do Produto
    1 * ln(x) + x * (1/x) = ln(x) + 1
    """
    if x <= 0: return 1e9
    return math.log(x) + 1

def phi_f21(x: float) -> float:
    """
    Função phi para o MPF.
    Isolando x: ln(x) = 1/x -> x = e^(1/x)
    """
    if x == 0: return x
    return math.exp(1/x)

# --- EXEMPLO 22 ---
# f(x) = x^3 - 3.5x^2 + 4x - 1.5
# Polinômio de grau 3

def f22(x: float) -> float:
    """Função exemplo 22: f(x) = x^3 - 3.5x^2 + 4x - 1.5"""
    return x**3 - 3.5*(x**2) + 4*x - 1.5

def df22(x: float) -> float:
    """Derivada f'(x) = 3x^2 - 7x + 4"""
    return 3*(x**2) - 7*x + 4

def phi_f22(x: float) -> float:
    """
    Função phi para o MPF.
    Isolando o termo linear 4x:
    4x = -x^3 + 3.5x^2 + 1.5
    x = (-x^3 + 3.5x^2 + 1.5) / 4
    """
    return (- (x**3) + 3.5*(x**2) + 1.5) / 4

In [ ]:
display(Markdown("### 1. Reprodução do Exemplo 18"))

### Estudo Especial de Equações Polinomiais